In [90]:
# Si no tenemos el módulo instalado
# !pip install google-cloud-secret-manager
!pip install langchain langchain-openai

### LangChain: Modelos, Prompts and Parsers de salida


In [91]:
# Primero autenticamos el usuario :
import sys
from google.colab import userdata
openai_token = userdata.get('OPENAI_TOKEN')


In [92]:
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

engine = "gpt-4"

model = ChatOpenAI(temperature=0.7, model=engine,openai_api_key=openai_token)


## Plantilla de Prompt

In [93]:
estilo_formal = """Español Castellano \
correcto, educado y conciliador
"""

In [94]:
plantilla = """Transforma el texto \
delimitado por acento grave triple \
en un texto con el estilo {estilo}. \
text: ```{texto}```
"""

In [95]:
from langchain.prompts import ChatPromptTemplate

prompt_template = ChatPromptTemplate.from_template(plantilla)
print(prompt_template.messages[0].prompt)
print(prompt_template.messages[0].prompt.input_variables)

input_variables=['estilo', 'texto'] template='Transforma el texto delimitado por acento grave triple en un texto con el estilo {estilo}. text: ```{texto}```\n'
['estilo', 'texto']


In [96]:
texto_original = """
Bueno, la verdad es que paso bastante de los resultados.
Me importa un bledo lo que piensen en dirección. La próxima vez que me contacten
para hablar de esto, les voy a mandar al carajo!
"""

In [97]:
messages = {
    "estilo": estilo_formal,
    "texto": texto_original
}


In [98]:
###Llamamos al LLM para transformar el mensaje según la plantilla
chain = prompt_template | model | StrOutputParser()
response = chain.invoke(messages)
response

'En realidad, no suelo prestar demasiada atención a los resultados. La opinión de la dirección no es algo que me afecte en particular. La próxima vez que me contacten para tratar este tema, les expresaré amablemente mi desinterés en continuar con la conversación.'

### Procesador de salida (output parsers)
Ayudan a transformar la salida cruda del modelo en algo más utilizable o en un formato específico que sea más fácil de manejar en la aplicación donde se utiliza el modelo.

In [99]:
{
  "difícil": False,
  "material": "Muy bueno!",
  "calidad_precio": "buena relación calidad-precio"
}

{'difícil': False,
 'material': 'Muy bueno!',
 'calidad_precio': 'buena relación calidad-precio'}

In [100]:
valoracion = """\
Me ha gustado mucho el curso. El material ha sido extenso e interesante.
A pesar de ello, era la primera vez que estudiaba estos conceptos
y me costó un poco asimilarlos.
Estoy muy contento con haber hecho esta inversión de dinero en mi conocimiento.
Lo recomendaré a otros colegas de trabajo!
"""

plantilla = """\
Del siguiente texto, extrae la siguiente información:

dificultad: El curso le resulto fácil o difícil
Contesta True si fue difícil, False si no lo fue o no se indica.

material: El material cumplió con sus expectativas?
Valoralo el sentimiento sobre el material del 1 al 5.
Si no se menciona nada sobre el material devuelve -1.

calidad_precio: Extrae cualquier referencia a la calidad o precio,\
y relacionalos ofreciendo el sentimiento.

Formatea la salida como JSON con las siguientes claves:
dificultad
material
calidad_precio

text: {text}
"""

In [101]:
from langchain.prompts import ChatPromptTemplate

prompt_plantilla = ChatPromptTemplate.from_template(plantilla)
prompt_plantilla

ChatPromptTemplate(input_variables=['text'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['text'], template='Del siguiente texto, extrae la siguiente información:\n\ndificultad: El curso le resulto fácil o difícil\nContesta True si fue difícil, False si no lo fue o no se indica.\n\nmaterial: El material cumplió con sus expectativas?\nValoralo el sentimiento sobre el material del 1 al 5.\nSi no se menciona nada sobre el material devuelve -1.\n\ncalidad_precio: Extrae cualquier referencia a la calidad o precio,y relacionalos ofreciendo el sentimiento.\n\nFormatea la salida como JSON con las siguientes claves:\ndificultad\nmaterial\ncalidad_precio\n\ntext: {text}\n'))])

In [102]:
messages = { 'text': valoracion }

chain = prompt_plantilla | model | StrOutputParser()

response = chain.invoke(messages)
print(response)

{
"dificultad": true,
"material": 4,
"calidad_precio": "Estoy muy contento con haber hecho esta inversión de dinero en mi conocimiento. Lo recomendaré a otros colegas de trabajo!"
}


In [103]:
from langchain.output_parsers import ResponseSchema
from langchain.output_parsers import StructuredOutputParser

In [104]:
dificultad_schema = ResponseSchema(name="dificultad", description="¿El curso le resulto fácil o difícil? Contesta True si fue difícil, False si no lo fue o no se indica.")
material_schema = ResponseSchema(name="material", description="El material cumplió con sus expectativas? Valoralo el sentimiento sobre el material del 1 al 5. Si no se menciona nada sobre el material devuelve -1.")
calidad_precio_schema = ResponseSchema(name="calidad_precio", description="Extrae cualquier referencia a la calidad o el precio y relaciónalos ofreciendo el sentimiento.")

response_schemas = [dificultad_schema,
                    material_schema,
                    calidad_precio_schema]

In [105]:
procesador_salida = StructuredOutputParser.from_response_schemas(response_schemas)

In [106]:
format_instructions = procesador_salida.get_format_instructions()

In [107]:
plantilla2 = """\
Del siguiente texto, extrae la siguiente información:

dificultad: El curso le resulto fácil o difícil. Contesta True si fue difícil, False si no lo fue o no se indica.

material: El material cumplió con sus expectativas? Valoralo el sentimiento sobre el material del 1 al 5. Si no se menciona nada sobre el material devuelve -1.

calidad_precio: Extrae cualquier referencia a la calidad o precio y relacionalos ofreciendo el sentimiento.

Formatea la salida como JSON con las siguientes claves:
dificultad
material
calidad_precio

text: {text}

{format_instructions}
"""

prompt = ChatPromptTemplate.from_template(template=plantilla2)

messages = prompt.format_messages(text=valoracion, format_instructions=format_instructions)

In [108]:
print(messages[0].content)

Del siguiente texto, extrae la siguiente información:

dificultad: El curso le resulto fácil o difícil. Contesta True si fue difícil, False si no lo fue o no se indica.

material: El material cumplió con sus expectativas? Valoralo el sentimiento sobre el material del 1 al 5. Si no se menciona nada sobre el material devuelve -1.

calidad_precio: Extrae cualquier referencia a la calidad o precio y relacionalos ofreciendo el sentimiento.

Formatea la salida como JSON con las siguientes claves:
dificultad
material
calidad_precio

text: Me ha gustado mucho el curso. El material ha sido extenso e interesante.
A pesar de ello, era la primera vez que estudiaba estos conceptos
y me costó un poco asimilarlos.
Estoy muy contento con haber hecho esta inversión de dinero en mi conocimiento.
Lo recomendaré a otros colegas de trabajo!


The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"dificultad": stri

In [109]:
response = chain.invoke(messages)

In [110]:
print(response)

```json
{
	"dificultad": true,
	"material": 4,
	"calidad_precio": "positivo"
}
```


In [111]:
output_dict = procesador_salida.parse(response)

In [112]:
output_dict

{'dificultad': True, 'material': 4, 'calidad_precio': 'positivo'}